In [15]:
import spacy
from tqdm import tqdm
from collections import defaultdict
from sklearn.pipeline import Pipeline
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
import numpy as np



class SentencesIterator:
    def __init__(self, path):
        self.path = path

    def __iter__(self):
        with open(self.path, 'r') as f:
            for l in f.readlines():
                yield l.strip()

In [2]:
sentences = SentencesIterator('xaa')
sents = list(sentences)

In [3]:
def tag_sent(sent):
    return '< < {} > >'.format(sent)

def processing(sents, disable_pipe=['parser', 'ner'], max_sents=50000):
    nlp = spacy.load('es_core_news_sm')
    tokens = []
    for sent in tqdm(sents[:max_sents]):
        if len(sent) > 7:
            tokens.append([token for token in nlp(tag_sent(sent), disable=disable_pipe)])
    return tokens
    
    
    
    

In [4]:
tokens = processing(sents, max_sents=300, disable_pipe=['ner'])

100%|██████████| 300/300 [00:02<00:00, 134.87it/s]


In [5]:


def make_feature_dict(doc):
    wordnet_lemmatizer = WordNetLemmatizer()
    document_features = []
    words = []
    for i, token in enumerate(doc):
        if token.text not in ['<', '>']:
            features = {
                'word': token.text,
                'pword': doc[i-1].text,
                'ppword': doc[i-2].text,
                'nword': doc[i+1].text,
                'nnword': doc[i+2].text,
                'pword_tag': doc[i-1].pos_,
                'ppword_tag': doc[i-2].pos_,
                'nword_tag': doc[i+1].pos_,
                'nnword_tag': doc[i+2].pos_,
                'is_lower': token.is_lower,
                'POS': token.pos_,
                'lemma': wordnet_lemmatizer.lemmatize(token.text, pos='v'),
                #'word_len': len(token.text),
                #'prob': token.prob,
                'is_stop': token.is_stop,
                'is_oov': token.is_oov,
            }
            document_features.append(features)
            words.append(token.text)
#             document_features_dict[token.text].append(features)
#             check_list.append(token.text)
    return document_features, words

In [6]:
tokens_f = [make_feature_dict(doc) for doc in tokens]
# tokens_f = [make_word_feature_dict(doc) for doc in tokens]
# make_feature_dict(tokens[0])

In [21]:
tokens_f = []
words = []
for doc in tokens:
    toks, words_ = make_feature_dict(doc) 
    tokens_f += toks
    words += words_    

In [22]:
tokens_f = np.array(tokens_f)

In [31]:
vectorizer_config = {
        'sparse': False
}
svd_config = {
        'n_components': 14,
        'random_state': 666,
        'n_iter': 15,
}
kmeans_config = {
        'n_clusters': 35,
        'random_state': 666,
        'init': 'random',
        'precompute_distances': False,
}


pipeline = Pipeline([
        ('vect', DictVectorizer(**vectorizer_config)),
        ('svd', TruncatedSVD(**svd_config)),
        ('kmeans', KMeans(**kmeans_config)),
])



In [32]:
pipeline

Pipeline(memory=None,
     steps=[('vect', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=False)), ('svd', TruncatedSVD(algorithm='randomized', n_components=14, n_iter=15,
       random_state=666, tol=0.0)), ('kmeans', KMeans(algorithm='auto', copy_x=True, init='random', max_iter=300,
    n_clusters=35, n_init=10, n_jobs=None, precompute_distances=False,
    random_state=666, tol=0.0001, verbose=0))])

In [33]:
labels = pipeline.fit_predict(tokens_f)

In [34]:
from collections import defaultdict
words_dict = defaultdict(set)
for i, label in enumerate(labels):
    words_dict[label].add(words[i])
words_dict = dict(words_dict)
# print(words)

In [35]:
for i in words_dict.values():
    print(i)

{'Situado', 'Viajan', 'Participa', 'Cooperstown', 'Treint', 'Aparecerían', 'Nacional', 'Retoma', 'Santiago', 'En', 'Origen', 'Finalizado', 'Defiende', 'Alfred', 'Cherry', 'Christian', 'Clasificaron', 'Dique', 'CGT', 'Creación', 'Milford', 'Miembro', 'Amanita', 'Fue', 'Suele', 'Alrededor', 'Final', 'Trotsky', 'Cuenta', 'Apoya', 'Además', 'Gilbertsville', 'Contó', 'Giovacchini', 'Keystone', 'Ejemplar', 'Fallece', 'Louis', 'Contrario', 'Monatte', 'Sinónimos', 'Copa', 'Igualmente', 'Va', 'Morris', 'Carrie', 'Otego', 'Traducción', 'Organiza', 'Rechazan', 'Laurens', 'Recibía', 'Habilitado', 'Destacan', 'Siempre', 'Swanzey', 'Richfield', 'Stevenson', 'Desaprobado', 'Hoy', 'Entra', 'Vuelve', 'Rosmer', 'Anima', 'Recibe'}
{'DIGITO', 'Springs', 'Cooperstown', 'Treint', 'Borís', 'Griot', 'Gallimard', 'Cardinals', 'Karl', 'Limacella', 'Alfred', 'París', 'Trotsky', 'Comité', 'Granada', 'Donovan', 'Ferrocarril', 'Kronstadt', 'Tirol', 'Créteil', 'Campeón', 'Zimmerwald', 'Lenin', 'VO', 'Monte', 'Keysto

In [68]:
# tokens_f[:3]

In [69]:
len(sents)

174650

In [16]:
import re, string, unicodedata
import nltk
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    return [word.lower() for word in words] 

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords.words('spanish'):
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = replace_numbers(words)
    words = remove_stopwords(words)
    return words

# words = normalize(words)

def stem_and_lemmatize(words):
    stems = stem_words(words)
    lemmas = lemmatize_verbs(words)
    return stems, lemmas
from tqdm import tqdm
lemma_sents = [lemmatize_verbs(sent) for sent in sents]
# print('Stemmed:\n', stems)
# print('\nLemmatized:\n', lemmas)

In [85]:
sents3=['-\n"Lo que sostiene a la pareja es el amor"\nClara Crespo (50) y Rodolfo Martínez (54) no se imaginan uno sin el otro.', '"Prefiero ni pensarlo", dice Clara.', 'Hace 26 años que están casados, y tienen cuatro hijas mujeres.', 'Se conocieron en el Ateneo Juventus, el movimiento juvenil de Capuchinos.', 'Hoy aseguran no estar sorprendidos del tiempo que llevan juntos sino de haber logrado entenderse tan bien.', '&#226;&#8364;&#8220;¿Qué les gusta y disgusta del otro?', '¿Qué quisieran cambiarle?', '&#226;&#8364;&#8220;Rodolfo: Me gusta que sea cariñosa, alegre y esté siempre pensando en mí, y que es una gran madre.', 'Me disgustaba que cuando se enojaba no quería hablar, pero ya no lo hace más.', 'A veces es indecisa pero ya me acostumbré.', 'No quiero cambiarle nada, que sea como es.', '&#226;&#8364;&#8220;Clara : Me gusta que es una persona emprendedora, alegre, optimista y servicial.', 'Me gustaría que a veces fuera más sutil para decir las cosas.', 'Pienso que las personas vamos cambiando con el tiempo de acuerdo a la edad, a las circunstancias que vivimos y todo lo que nos rodea.', 'Seguramente que no somos los mismos que cuando nos casamos y siempre seguiremos descubriendo cosas nuevas del otro.', 'Lo bueno es conversar y ayudarse a cambiar esas cosas que molestan al otro.', '&#226;&#8364;&#8220;¿Cuál fue el momento más difícil?', '-C: Sin duda fue cuando perdimos una hija.', 'Después de un dolor tan grande uno ve la vida de otra manera y ningún problema te parece tan grande.', 'Lo que más nos ayudó es que los dos compartimos la misma fe en Dios y sólo a través de &#195;&#8240;l podes darle otro sentido a la muerte.', 'También cuando vivimos separados un año y medio, por razones de trabajo de Rody.', 'Clara asegura que lo que sostiene a la pareja es el amor.', '"Hay que alimentarlo para que crezca siempre.', 'Los proyectos y objetivos en común también ayudan a tener ilusiones y ganas de seguir juntos, pero todo es inútil si no hay amor.', 'Hay que tratar de crecer en todos los aspectos en forma permanente".', '&#226;&#8364;&#8220;Un matrimonio a largo plazo, ¿es un refugio contra inseguridades, una rareza, un triunfo, orgullo?', '&#226;&#8364;&#8220;R: Es un medio para ser feliz, un proyecto de vida.', '&#226;&#8364;&#8220;C: Es una hermosa experiencia, más que todo eso.', 'El triunfo es ir logrando quererse cada día más.', '&#226;&#8364;&#8220;¿Por qué ahora las parejas duran menos?', '&#226;&#8364;&#8220;C: Creo que puede faltar comunicación y a veces proyecto y objetivos en común.', 'Cuando uno elige la vida de a dos a veces tiene que dejar de lado o postergar intereses personales.', '-¿Los recursos económicos son un conflicto?', '&#226;&#8364;&#8220;R: No son un problema, aunque a veces no había suficiente nunca fue una prioridad.', 'En general lo manejamos juntos, aunque el día a día lo lleva Clara.', '&#226;&#8364;&#8220;C: Siempre pusimos en común los ingresos cuando los dos trabajábamos.', 'Todo es de los dos.', '¡Menos mal, sino ahora que no trabajo estaría chau!', 'Y nos ponemos de acuerdo en la forma de administrarlo.', '&#226;&#8364;&#8220;¿La pasión es el secreto de la duración feliz?', 'R: La pasión es necesaria, pero no es el secreto de la felicidad.', 'Es importante mantener la pasión de los primeros años, toda la vida.', '-\nEl Carbó y el Ipem 270 levantaron la toma\nLos secundarios Ipem 270 Manuel Belgrano y Alejandro Carbó decidieron ayer levantar la toma de las instituciones escolares.', 'Así, se sumaron a la medida que ya había tomado el Jerónimo Luis de Cabrera el sábado.', 'Estos tres colegios habían iniciado la toma el miércoles 29 de septiembre en reclamo de mejoras edilicias y pidiendo que se discuta el anteproyecto de reforma de la ley de Educación.', 'El sábado, suscribieron un acuerdo con el Ministerio de Educación por el que levantaban las tomas a cambio de planes de obras de las y una instancia de debate para la normativa.', '"Si el Gobierno nos toma el pelo, volveremos a las tomas", advirtieron desde Secundarios Unidos de Córdoba, que aglutina a los centros de estudiantes de esas escuelas, al tiempo que calificaron lo logrado como "una victoria".', '"Que sepan que cada día somos más y más colegios los que abrimos los ojos para luchar día a día por una educación para todos y todas", agregan.', 'También firmaron el acuerdo el Deán Funes, el Nicolás Copérnico y el Ipem 16 de Villa Cornú, que en principio levantarían las tomas entre martes y miércoles, cuando lleguen los planes de obras.', 'No obstante, desde el grupo que mantiene las medidas de fuerza por el reclamo de la ley de Educación pusieron en duda esa posibilidad.', 'Ese grupo, denominado Coordinadora Interestudiantil, también volvió a llamar a la unidad del movimiento estudiantil.', '"Convocamos a las escuelas que han firmado el acta porque se han empezado algunos de los planes de refacción a continuar las luchas", sostuvieron.', '-\n"Tenemos una familia hermosa, qué más pedir"\nPedro (78) y Mary (74) parecen estar viviendo una luna de miel, pero llevan 51 años casados.', 'Pedro la trata como a una reina, de vez en cuando le compra bombones o la sorprende con una carta que algún locutor lee en la radio.', 'La llama "Gordita" a cada rato, la abraza y se ríen.', 'No han tenido una vida fácil, dicen.', 'Pero agradecen lo que les ha dado: cinco hijos (uno murió) y 13 nietos.', '"Estamos juntos hace más de 21 mil días.', 'Hace 59 años que estamos de novios", comenta con precisión Pedro Rodríguez, en su casa de barrio Ayacucho.', 'En la entrada del hogar que habitan hace 46 años, hay una Virgen y una leyenda que anticipa los cimientos con los que se construyó este hogar.', 'Dice más o menos así: "Somos Pedro y Mary.', 'Tenemos una familia hermosa, qué más podemos pedir".', 'Mary es María Isabel Barrionuevo, ex empleada de la Fábrica Militar de Aviones.', 'La mujer cuenta que se conocieron a los 13 años y todo lo que vino después.', 'La charla es tan amena que el agua para el café se consume por completo.', '"Nada que ver con las relaciones de ahora.', 'A los dos meses de estar de novios, recién me dio el primer beso.', 'Yo lo paraba en seco", recuerda Mary.', 'Sin embargo no cree que el tiempo pasado haya sido mejor.', '"Había muchos tabúes".', '&#226;&#8364;&#8220;¿Se imaginaron que iban a durar tanto tiempo?', '&#226;&#8364;&#8220;Uno no tuvo tiempo de analizar.', 'Nos casamos enamorados.', '¡No sabés lo que fue la luna de miel!', '¡Salir de noche solos!', 'Nunca habíamos salido solos&#226;&#8364;&#8220;, dice Mary, entre risas.', 'Coinciden en que son muy compañeros y saben conversar.', 'Todo lo hacen juntos.', '"Pobrecito el que se quede cuando el otro desaparezca", reflexiona Pedro, ex empleado de Entel.', '"Mi madre me decía siempre que cuando en una discusión uno está nervioso, el otro se debe callar.', 'Hay que esperar que pare la tormenta y después hablar.', 'A nosotros nos ayudó", aconseja Mary.', 'Pedro asegura que no han tenido tiempo para peleas; si había discusiones, era por los hijos.', '"A veces uno salía en defensa de uno o de otro", admiten.', 'Creen que los momentos más difíciles fueron los comienzos.', 'Los otros avatares de la vida los unió más.', '&#226;&#8364;&#8220;¿Qué les gusta del otro?', '&#226;&#8364;&#8220;&#195;&#8240;l es muy noble, honesto, siempre está tratando de ayudar, es sencillo y generoso, dice Mary, con mucha seguridad.', 'Pedro la mira, y responde: "Me gusta todo, es inigualable".', '-']

In [113]:
from nltk import pos_tag, word_tokenize

sents2 = [pos_tag(word_tokenize(sent)) for sent in sents3]

In [114]:
def make_feature_dict(base_feats, nf, sent, i):
    feat_dict = {}
    for n in range(0, nf + 1):
        for feature, fun in base_feats.items():
            prev = "p"*n
            nxt = "n"*n
            if len(sent) <= n + i:
                continue
            feat_dict[prev + feature] = fun(sent[i-n])
            feat_dict[nxt + feature] = fun(sent[i+n])

    return feat_dict


def feature_dict(sent, i, n=3):
    # n must be odd
    if n % 2 != 1:
        n -= 1

    if "<s>" not in sent:
        sent = ["<s>"] + list(sent) + ["</s>"]
        i += 1

    n_feats = int(n/2)

    base_feats = {
            "w": str.lower,
            "wu": str.isupper,
            "wt": str.istitle,
            "wd": str.isdigit,
        }

    return make_feature_dict(base_feats, n_feats, sent, i)

In [103]:
check_list = []
features = []
for sent in sents2:
    for i, (fst, snd) in enumerate(sent):
        new_stuff_dict = feature_dict()
        features.append(new_stuff_dict)
        check_list.append(fst)

In [104]:

from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=True)
X = v.fit_transform(features)

In [105]:
X

<1647x576 sparse matrix of type '<class 'numpy.float64'>'
	with 4941 stored elements in Compressed Sparse Row format>

In [106]:
Y = v.inverse_transform(X)


In [107]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=78, n_iter=7, random_state=42)
svd.fit_transform(X)

array([[ 2.30140056e-15, -1.27194493e-14,  2.30552124e-15, ...,
         2.14722387e-03, -4.19240582e-04,  2.52716197e-04],
       [ 1.68415460e-16,  2.32322051e-15, -1.49428344e-15, ...,
        -6.67346902e-12, -2.71887895e-14, -1.10195775e-12],
       [ 1.51029456e-02,  1.68153732e-02,  1.37435387e-14, ...,
        -1.91517363e-03,  8.92896837e-03,  1.39674278e-02],
       ...,
       [ 1.68431321e-16,  3.05719468e-17,  2.05335165e-16, ...,
        -6.67353163e-12, -2.73215076e-14, -1.10212431e-12],
       [-1.25048762e-16,  2.06024294e-14,  1.41155365e+00, ...,
         7.96165703e-04,  1.53135684e-04, -1.47149179e-03],
       [ 2.98838002e-15,  4.40963587e-15,  2.50578602e-15, ...,
         2.14722387e-03, -4.19240582e-04,  2.52716197e-04]])

In [108]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters=40)
km.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=40, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [109]:
labels = km.predict(X)

In [110]:
labels

array([37, 22,  7, ..., 22,  3, 37], dtype=int32)

In [112]:
from collections import defaultdict
words = defaultdict(set)
for i, label in enumerate(labels):
    words[label].add(check_list[i])
words = dict(words)
# for key, value in words.items():
#     print('CLuster:', key)
#     print(list(value)[:10])
words

{37: {'-'},
 22: {"''"},
 7: {')',
  'Alejandro',
  'Así',
  'Ateneo',
  'Aviones',
  'Ayacucho',
  'Barrionuevo',
  'Belgrano',
  'C',
  'Cabrera',
  'Capuchinos',
  'Carbó',
  'Clara',
  'Coinciden',
  'Convocamos',
  'Coordinadora',
  'Copérnico',
  'Cornú',
  'Creen',
  'Creo',
  'Crespo',
  'Cuando',
  'Córdoba',
  'Después',
  'Deán',
  'Dice',
  'Dios',
  'Educación',
  'El',
  'En',
  'Entel',
  'Es',
  'Ese',
  'Estamos',
  'Estos',
  'Funes',
  'Fábrica',
  'Gobierno',
  'Gordita',
  'Había',
  'Hace',
  'Hay',
  'Hoy',
  'Interestudiantil',
  'Ipem',
  'Isabel',
  'Jerónimo',
  'Juventus',
  'La',
  'Lo',
  'Los',
  'Luis',
  'Manuel',
  'Martínez',
  'Mary',
  'María',
  'Me',
  'Mi',
  'Militar',
  'Ministerio',
  'Nada',
  'Nicolás',
  'Nos',
  'Nunca',
  'Pero',
  'Pienso',
  'Pobrecito',
  'Prefiero',
  'Que',
  'R',
  'Rodolfo',
  'Rodríguez',
  'Rody',
  'Se',
  'Secundarios',
  'Seguramente',
  'Si',
  'Siempre',
  'Sin',
  'Somos',
  'También',
  'Tenemos',
  'Todo'

In [64]:
print("Top terms per cluster:")
# order_centroids = model.cluster_centers_.argsort()[:, ::-1]
for i in range(40):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

Top terms per cluster:
Cluster 0:
 w=que
 nw=a
 w=las
 pw=de
 pw=asimismo
 pw=problemas
 nw=un
 pw=recomendación
 nw=situación
 pw=resuelva
Cluster 1:
 wt
 w=subsecretario
 nwt
 nw=general
 pw=al
 pw=digito
 pw=además
 nw=respecto
 nw=situación
 nw=tema
Cluster 2:
 w=casos
 nw=por
 pw=digito
 pwu
 wu
 pw=<s>
 nw=respecto
 nw=situación
 nw=tema
 nw=traslado
Cluster 3:
 w=categorías
 pw=las
 nw=de
 wu
 pw=administrativas
 nw=respecto
 nw=situación
 nw=tema
 nw=traslado
 nw=un
Cluster 4:
 wu
 pwu
 pw=digito
 w=digito
 nw=digito
 nwu
 pw=además
 nw=respecto
 nw=situación
 nw=tema
Cluster 5:
 pw=consiguiente
 nw=el
 w=persistiría
 wu
 pw=además
 nw=reinserción
 nw=respecto
 nw=situación
 nw=tema
 nw=traslado
Cluster 6:
 pwu
 pw=digito
 nw=número
 w=del
 wu
 pw=<s>
 nw=respecto
 nw=situación
 nw=tema
 nw=traslado
Cluster 7:
 nw=de
 pw=la
 w=función
 w=utilización
 pw=<s>
 nwu
 nwt
 nwd
 nw=vigentes
 wu
Cluster 8:
 wt
 nw=lo
 w=por
 pw=<s>
 pw=administrativas
 nw=respecto
 nw=situación
 nw=te